# Build Excel files

Refer to the [Tally documentation on building Excel files](https://python.datasmoothie.com/7_build_excel_output.html) for tips on what can be done here.

In [16]:
import tally
import os
import pandas as pd
import json
ds = tally.DataSet(api_key=os.environ.get("TALLY_KEY"))

In [ ]:
project_name = "tally_example_project"

In [17]:
ds.use_quantipy(f"../data/4_{project_name}.json",f"../data/4_{project_name}.csv")

# Build starts here

## 1. Define your banner variables (to appear at the top of your file)

In [18]:
# add your banner variables here, e.g. gender, age, etc
table_banner = []

#### this only runs if you haven't specified banner variables yourself - we get some common variable names

In [19]:
if len(table_banner) == 0:
  variables = ds.variables()

  with open('../specs/common_variable_names.json', 'r') as f:
      common_variables = json.load(f)
  common_variables = [i.lower() for i in common_variables]
  for var in variables['single']:
    if any(match in var for match in common_variables):
      table_banner.append(var)

# 2. Define stub/side variables

These will apear on the left.

In [20]:
# add your question list, e.g.
# question_list = ['q1',
#  'q2',
#  'q3',
#  'q4',
#  'q5'
#]

question_list = []

#### if you haven't defined the list, we add all questions except the banner questions

In [ ]:
if len(question_list) == 0:
  question_list = variables['single']
  question_list = [i for i in question_list if i not in table_banner]

# 3. Build the Excel table

In [22]:
build = tally.Build(name='client A', default_dataset=ds)

####
# Set options sthat apply to whole build/Excel file
####
# set weight
#build.options.set_weight('weight')
# set base outside of main table
build.options.set_base_position('outside')
# brand with font
build.options.set_font('open sans')
# only show percentage results
build.options.set_ci(['c%'])

sheet = build.add_sheet(banner=table_banner)

####
# Set options that apply on a sheet-level
####

# set color and weight of base
sheet.options.set_answer_format(
    'base', 
    {"font_color":"F15A30", "bold":True,'text_wrap': True}
)
sheet.options.set_question_format('percentage', {"bold":True})
sheet.options.set_column_format_for_type('base', 1, {"bold":True})
sheet.options.set_column_format_for_type('percentage', 1, {"bold":True})

####
# Run the calculations and add them to a build
####
for questionlist_index, question in enumerate(question_list):
    if questionlist_index ==0:
        sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})
    else:
        sheet.options.set_base_position('hide')
        sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})


In [23]:
build.save_excel(f'../outputs/{project_name}.xlsx')

In [24]:
ds.write_quantipy(f"../data/5_{project_name}.json",f"../data/5_{project_name}.csv")